In [17]:
import pandas as pd
from datetime import datetime, time, timedelta

In [18]:
def find_ids_within_ten_percentage_threshold(df, reference_id)->pd.DataFrame():
    # Calculate the average distance for the reference_id
    reference_avg_distance = df[df['id_start'] == reference_id]['distance'].mean()

    # Calculate the threshold range (10% of the average distance)
    threshold = 0.1 * reference_avg_distance

    # Filter rows where the distance is within the threshold range
    within_threshold_df = df[(df['id_start'] != reference_id) & 
                             (df['distance'] >= reference_avg_distance - threshold) & 
                             (df['distance'] <= reference_avg_distance + threshold)]

    # Extract unique IDs from the result
    result_ids = within_threshold_df['id_start'].unique()

    # Sort the result IDs
    result_ids = sorted(result_ids)

    # Create a DataFrame with the result IDs
    result_df = pd.DataFrame({'id_start': result_ids})

    return result_df

In [22]:
def calculate_time_based_toll_rates(df)->pd.DataFrame():
    df['start_day'] = df['start_time'].dt.strftime('%A')
    df['start_time'] = df['start_time'].dt.time
    df['end_day'] = df['end_time'].dt.strftime('%A')
    df['end_time'] = df['end_time'].dt.time

    df['discount_factor'] = 1.0

    # Apply discount factor based on time range
    def apply_discount_factor(row):
        if row['start_day'] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            if row['start_time'] >= time(0, 0, 0) and row['start_time'] < time(10, 0, 0):
                return 0.8
            elif row['start_time'] >= time(10, 0, 0) and row['start_time'] < time(18, 0, 0):
                return 1.2
            elif row['start_time'] >= time(18, 0, 0) and row['start_time'] <= time(23, 59, 59):
                return 0.8
        elif row['start_day'] in ['Saturday', 'Sunday']:
            return 0.7
        return 1.0

    df['discount_factor'] = df.apply(apply_discount_factor, axis=1)

    return df

In [23]:
df = pd.read_csv('dataset-3.csv')
reference_id = 1001400
result_df = find_ids_within_ten_percentage_threshold(df, reference_id)
print(result_df)
df_with_time_based_toll_rates = calculate_time_based_toll_rates(df[df['id_start'].isin(result_df['id_start'])])
print(df_with_time_based_toll_rates)

   id_start
0   1001420
1   1001430
2   1001436
3   1001438
4   1001446
5   1001450
6   1001456
7   1001468


KeyError: 'start_time'